In [1]:
!pip install tensorflow==2.12 keras==2.12 larq

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.0 MB/s eta 0:00:00
   ━━━

In [1]:
# Row Inversion

import tensorflow as tf
import larq as lq
import numpy as np
import os

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(-1, 784).astype("float32") / 127.5 - 1
test_images = test_images.reshape(-1, 784).astype("float32") / 127.5 - 1

train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

11490434/11490434 [==============================] - 0s 0us/step


In [2]:
# Defining binary quantization params
kwargs = dict(
    input_quantizer="ste_sign",
    kernel_quantizer="ste_sign",
    kernel_constraint="weight_clip",
    use_bias=False
)

# BNN model
model = tf.keras.models.Sequential([
    lq.layers.QuantDense(512, input_shape=(784,), **kwargs),
    tf.keras.layers.BatchNormalization(),

    lq.layers.QuantDense(512, **kwargs),
    tf.keras.layers.BatchNormalization(),

    lq.layers.QuantDense(512, **kwargs),
    tf.keras.layers.BatchNormalization(),

    lq.layers.QuantDense(10, **kwargs),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("softmax")
])

In [3]:
# Compiling and training model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_data=(test_images, test_labels))

# Evaluating original model
print("\nOriginal Model Accuracy:")
orig_loss, orig_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {orig_acc * 100:.2f}%")

Epoch 1/10
469/469 [==============================] - 26s 46ms/step - loss: 0.5578 - accuracy: 0.8940 - val_loss: 0.4277 - val_accuracy: 0.9049
Epoch 2/10
469/469 [==============================] - 22s 47ms/step - loss: 0.2708 - accuracy: 0.9471 - val_loss: 0.2473 - val_accuracy: 0.9453
Epoch 3/10
469/469 [==============================] - 18s 38ms/step - loss: 0.1885 - accuracy: 0.9597 - val_loss: 0.1844 - val_accuracy: 0.9565
Epoch 4/10
469/469 [==============================] - 19s 41ms/step - loss: 0.1455 - accuracy: 0.9666 - val_loss: 0.1727 - val_accuracy: 0.9531
Epoch 5/10
469/469 [==============================] - 18s 38ms/step - loss: 0.1202 - accuracy: 0.9705 - val_loss: 0.1466 - val_accuracy: 0.9596
Epoch 6/10
469/469 [==============================] - 19s 41ms/step - loss: 0.0998 - accuracy: 0.9744 - val_loss: 0.1393 - val_accuracy: 0.9614
Epoch 7/10
469/469 [==============================] - 18s 38ms/step - loss: 0.0849 - accuracy: 0.9780 - val_loss: 0.1497 - val_accuracy:

In [4]:
# Function to binarize weights
def get_binarized_weights(layer):
    weights = layer.get_weights()[0]
    quantizer = layer.kernel_quantizer
    return quantizer(weights).numpy()

# Printing binarized weights before transformation
print("\nBinarized Weights Before Row Inversion:")
for layer in model.layers:
    if isinstance(layer, lq.layers.QuantDense):
        print(f"\nLayer: {layer.name}")
        print(get_binarized_weights(layer)[:10, :10])


Binarized Weights Before Row Inversion:

Layer: quant_dense
[[-1. -1. -1.  1.  1.  1.  1. -1.  1. -1.]
 [ 1. -1.  1. -1.  1.  1. -1. -1. -1.  1.]
 [ 1. -1.  1.  1. -1. -1.  1. -1.  1.  1.]
 [-1. -1. -1.  1. -1. -1.  1. -1.  1. -1.]
 [ 1. -1. -1. -1. -1.  1.  1. -1.  1. -1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1. -1.  1.]
 [-1. -1.  1.  1. -1.  1. -1.  1. -1.  1.]
 [-1. -1. -1.  1.  1. -1.  1.  1. -1.  1.]
 [ 1. -1. -1. -1. -1.  1.  1. -1.  1.  1.]
 [-1.  1. -1. -1. -1.  1.  1. -1.  1.  1.]]

Layer: quant_dense_1
[[ 1. -1. -1. -1.  1. -1. -1. -1. -1. -1.]
 [-1. -1.  1. -1.  1.  1.  1. -1. -1.  1.]
 [-1.  1.  1.  1.  1.  1.  1. -1.  1. -1.]
 [-1.  1. -1.  1. -1.  1. -1.  1. -1.  1.]
 [ 1. -1.  1. -1.  1. -1. -1. -1.  1.  1.]
 [ 1. -1.  1.  1. -1. -1.  1. -1.  1. -1.]
 [ 1.  1.  1. -1. -1.  1. -1.  1. -1.  1.]
 [ 1.  1.  1. -1.  1. -1. -1. -1.  1.  1.]
 [-1.  1. -1.  1.  1.  1. -1.  1. -1. -1.]
 [ 1. -1.  1. -1.  1.  1.  1.  1. -1. -1.]]

Layer: quant_dense_2
[[-1.  1. -1.  1. -1. -1.  1. -1.

In [5]:
# Row inversion
def apply_row_inversion(model):
    for layer in model.layers:
        if isinstance(layer, lq.layers.QuantDense):
            weights = layer.get_weights()
            if weights:
                W = weights[0]
                rows = W.shape[0]
                row_puf = np.random.randint(0, 2, size=rows)
                W_new = np.copy(W)
                for r in range(rows):
                    if row_puf[r] == 1:
                        W_new[r, :] *= -1
                layer.set_weights([W_new])
    print("\n Row Inversion Applied.")

# Applying inversion
apply_row_inversion(model)


 Row Inversion Applied.


In [6]:
# Printing binarized weights after inversion
print("\nBinarized Weights After Row Inversion:")
for layer in model.layers:
    if isinstance(layer, lq.layers.QuantDense):
        print(f"\nLayer: {layer.name}")
        print(get_binarized_weights(layer)[:10, :10])

# Evaluating model
print("\nModel Accuracy After Row Inversion:")
inv_loss, inv_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy After Row Inversion: {inv_acc * 100:.2f}%")


Binarized Weights After Row Inversion:

Layer: quant_dense
[[ 1.  1.  1. -1. -1. -1. -1.  1. -1.  1.]
 [-1.  1. -1.  1. -1. -1.  1.  1.  1. -1.]
 [-1.  1. -1. -1.  1.  1. -1.  1. -1. -1.]
 [-1. -1. -1.  1. -1. -1.  1. -1.  1. -1.]
 [-1.  1.  1.  1.  1. -1. -1.  1. -1.  1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1. -1.  1.]
 [-1. -1.  1.  1. -1.  1. -1.  1. -1.  1.]
 [ 1.  1.  1. -1. -1.  1. -1. -1.  1. -1.]
 [-1.  1.  1.  1.  1. -1. -1.  1. -1. -1.]
 [ 1. -1.  1.  1.  1. -1. -1.  1. -1. -1.]]

Layer: quant_dense_1
[[-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.]
 [-1. -1.  1. -1.  1.  1.  1. -1. -1.  1.]
 [-1.  1.  1.  1.  1.  1.  1. -1.  1. -1.]
 [ 1. -1.  1. -1.  1. -1.  1. -1.  1. -1.]
 [-1.  1. -1.  1. -1.  1.  1.  1. -1. -1.]
 [-1.  1. -1. -1.  1.  1. -1.  1. -1.  1.]
 [ 1.  1.  1. -1. -1.  1. -1.  1. -1.  1.]
 [-1. -1. -1.  1. -1.  1.  1.  1. -1. -1.]
 [ 1. -1.  1. -1. -1. -1.  1. -1.  1.  1.]
 [ 1. -1.  1. -1.  1.  1.  1.  1. -1. -1.]]

Layer: quant_dense_2
[[ 1. -1.  1. -1.  1.  1. -1.  1. 

In [7]:
# Saving the inverted model
os.makedirs("model_row_inverted", exist_ok=True)
model.save("model_row_inverted/binarized_model_row_inverted", include_optimizer=False)

In [8]:
import shutil
from google.colab import files

#Zip the saved model folders
shutil.make_archive("/content/model_row_inverted.zip", 'zip', '/content/model_row_inverted')


'/content/model_row_inverted.zip.zip'